In [2]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict,Annotated
from langchain_core.messages import BaseMessage,HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver
import os

In [3]:
load_dotenv()
AK=os.getenv("GOOGLE_API_KEY")
llm = ChatGoogleGenerativeAI(
    model="gemini-3-flash-preview",
    google_api_key=AK
)


In [4]:
class JokeState(TypedDict):
    topic:str
    joke:str
    explanation:str
    


In [5]:
def genrate_joke(state: JokeState):
    prompt=f'gnrate a joke on topic {state["topic"]}'
    response=llm.invoke(prompt).content
    return {"joke":response}

In [6]:
def genrate_explanation(state:JokeState):
    prompt=f'write explanation for joke {state["joke"]}'
    response=llm.invoke(prompt).content
    return {"explanation":response}

In [7]:
graph=StateGraph(JokeState)

graph.add_node('genrate_joke',genrate_joke)
graph.add_node('genrate_explanation',genrate_explanation)

graph.add_edge(START,'genrate_joke')
graph.add_edge('genrate_joke','genrate_explanation')
graph.add_edge('genrate_explanation',END)

In [8]:
checkpointer=InMemorySaver()
workflow=graph.compile(checkpointer=checkpointer)

In [9]:
config1={"configurable":{"thread_id":1}}
workflow.invoke({"topic":"shimla"},config=config1)

{'topic': 'shimla',
 'joke': [{'type': 'text',
   'text': 'Here are a few jokes about Shimla, depending on what kind of humor you like:\n\n**The Punny One:**\nWhy did the bell pepper move to Himachal Pradesh?\nBecause it wanted to be a **Shimla Mirch**!\n\n**The "Fitness" One:**\nA tourist in Shimla asked a local: "Is the Mall Road far from here?"\nThe local replied: "It’s only 5 minutes if you’re walking down, but it\'s about 3 business days if you’re walking back up!"\n\n**The Monkey One:**\nA tourist was sitting on a bench in Shimla eating a sandwich. A monkey jumped down, grabbed his expensive sunglasses, and sat on a tree branch.\nThe tourist started pleading: "Please give them back! I\'ll give you my sandwich!"\nThe monkey looked at the sandwich, then at the sunglasses, and said: "Keep the bread, pal. I’m selling these on OLX to buy a jacket. It’s freezing up here!"\n\n**The Honeymoon One:**\nWhy is it so hard to take a landscape photo in Shimla?\nBecause no matter where you poin

In [ ]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'shimla', 'joke': [{'type': 'text', 'text': 'Here are a few jokes about Shimla, depending on what kind of humor you like:\n\n**The Punny One:**\nWhy did the bell pepper move to Himachal Pradesh?\nBecause it wanted to be a **Shimla Mirch**!\n\n**The "Fitness" One:**\nA tourist in Shimla asked a local: "Is the Mall Road far from here?"\nThe local replied: "It’s only 5 minutes if you’re walking down, but it\'s about 3 business days if you’re walking back up!"\n\n**The Monkey One:**\nA tourist was sitting on a bench in Shimla eating a sandwich. A monkey jumped down, grabbed his expensive sunglasses, and sat on a tree branch.\nThe tourist started pleading: "Please give them back! I\'ll give you my sandwich!"\nThe monkey looked at the sandwich, then at the sunglasses, and said: "Keep the bread, pal. I’m selling these on OLX to buy a jacket. It’s freezing up here!"\n\n**The Honeymoon One:**\nWhy is it so hard to take a landscape photo in Shimla?\nBecause no matt

In [ ]:
list(workflow.get_state_history(config1))  #history per each node

[StateSnapshot(values={'topic': 'shimla', 'joke': [{'type': 'text', 'text': 'Here are a few jokes about Shimla, depending on what kind of humor you like:\n\n**The Punny One:**\nWhy did the bell pepper move to Himachal Pradesh?\nBecause it wanted to be a **Shimla Mirch**!\n\n**The "Fitness" One:**\nA tourist in Shimla asked a local: "Is the Mall Road far from here?"\nThe local replied: "It’s only 5 minutes if you’re walking down, but it\'s about 3 business days if you’re walking back up!"\n\n**The Monkey One:**\nA tourist was sitting on a bench in Shimla eating a sandwich. A monkey jumped down, grabbed his expensive sunglasses, and sat on a tree branch.\nThe tourist started pleading: "Please give them back! I\'ll give you my sandwich!"\nThe monkey looked at the sandwich, then at the sunglasses, and said: "Keep the bread, pal. I’m selling these on OLX to buy a jacket. It’s freezing up here!"\n\n**The Honeymoon One:**\nWhy is it so hard to take a landscape photo in Shimla?\nBecause no mat

Time Travel

In [12]:
workflow.get_state({"configurable":{"thread_id":"1","checkpoint_id":"1f0def2d-1b5a-6922-8000-4613bf09678b"}})

StateSnapshot(values={'topic': 'shimla'}, next=('genrate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f0def2d-1b5a-6922-8000-4613bf09678b'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-12-22T04:58:13.028065+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0def2d-1b58-6c52-bfff-335a8299160f'}}, tasks=(PregelTask(id='504ef383-a0e0-5034-969c-06c18bec2136', name='genrate_joke', path=('__pregel_pull', 'genrate_joke'), error=None, interrupts=(), state=None, result={'joke': [{'type': 'text', 'text': 'Here are a few jokes about Shimla, depending on what kind of humor you like:\n\n**The Punny One:**\nWhy did the bell pepper move to Himachal Pradesh?\nBecause it wanted to be a **Shimla Mirch**!\n\n**The "Fitness" One:**\nA tourist in Shimla asked a local: "Is the Mall Road far from here?"\nThe local replied: "It’s only 5 minutes if you’re walking down, but it\'s about 3 business days if you’r

In [13]:
workflow.invoke(None,{'configurable':{"thread_id":"1","checkpoint_id":"1f0def2d-1b5a-6922-8000-4613bf09678b"}})  #inital state none 

{'topic': 'shimla',
 'joke': [{'type': 'text',
   'text': 'Here is a joke about Shimla:\n\nA man from the scorching heat of Delhi went to Shimla for a vacation. After walking around the Mall Road for an hour in the freezing cold, he saw a local man sitting comfortably in just a light sweater.\n\nThe tourist, shivering uncontrollably, asked him, "Bhaisahab, doesn’t it get cold here? My teeth are literally playing the drums!"\n\nThe local man smiled and said, "Sir, in Shimla, we have three stages of winter."\n\n"What are they?" asked the tourist.\n\nThe local man replied:\n1. **Stage 1:** You wear a jacket.\n2. **Stage 2:** You wear two jackets.\n3. **Stage 3:** You stop bathing and just tell everyone you have \'inner peace\'!"',
   'extras': {'signature': 'Eu4YCusYAXLI2nyJI5xBiRoCehKBdccpKwKbAeJxyb7hEuzrG9pxCDbF8v2/jlsj/ZThnziRD94WQfNkBIBHnI7LR0w9kjln2/bqlXOpDlHmRg8LrAQp35P/uLKp1XAAD5+VSqanSq31k/r4b+h65XIRGhL9yZMcR7uVAqFh3eTuZLLPQFR5tMGy5R0IOPbSgvdZIzqR4laJsLS7oIVY9nleFYMQyO15XrBnEvghe6